# Rotate vectors

In bubble bouncing simulations, we typically set the coordinate system in such a way that the $xz$ plane is the bouncing surface, which is typically tilted w.r.t. gravity / bubble velocity. In this case, it is required that the computed Oseen wake flow to be rotated in space properly to match the settings of the coordinate system. 

In this notebook, we implement helper functions that rotate vectors.

## 0 Packages

In [68]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt

## 1 Rodrigues' rotation formula

The rotation of a vector $\vec{v}$ to another vector $\vec{v'}$ has to be about an axis $\vec{k}$, whose orientation can be found by $\vec{v}\times\vec{v'}$. The rotation matrix $R$ that can convert $\vec{v}$ to $\vec{v'}$ can be found using the Rodriges' rotation formula:

$$
\mathbf{R} = \mathbf{I} \cos\theta + \mathbf{K} \sin\theta + \mathbf{k}\otimes\mathbf{k}^T(1-\cos\theta)
$$

where $\mathbf{I}$ is identity matrix, $\theta$ is the angle of rotation, $\mathbf{K}$ is a skey-symmetric matrix that is equivalent to the cross-product, satisfying $\mathbf{k}\times \mathbf{v} = \mathbf{Kv}$, and $v$ is the vector to be rotated.

$$
\mathbf{K} = \begin{pmatrix}
0 & -k_z & k_y \\
k_z & 0 & -k_x \\
-k_y & k_x & 0
\end{pmatrix}
$$

In [86]:
def _skew_symmetric(k):
    """
    Convert a vector k to a skew-symmetric matrix.
    
    Parameters
    ----------
    k : array_like
        A 3D vector.
    
    Returns
    -------
    K : ndarray
        A 3x3 skew-symmetric matrix corresponding to the vector k.
    """

    if np.isclose(np.linalg.norm(k), 1.0) == False:
        k = k / np.linalg.norm(k)
        
    K = np.array([[0.   , -k[2], k[1] ],
                    [k[2] , 0.   , -k[0]],
                    [-k[1], k[0] , 0.   ]])
    return K

In [ ]:
def rotation_matrix_axis(k, angle):
    """
    Rotate a vector v around an axis k by a given angle. This function computes the rotation matrix using the Rodrigues' rotation formula.
    
    Parameters
    ----------
    v : array_like
        A 3D vector to be rotated.
    k : array_like
        A 3D vector representing the axis of rotation.
    angle : float
        The angle in radians by which to rotate the vector.
    
    Returns
    -------
    v_rotated : ndarray
        The rotated vector.
    """

    if np.isclose(np.linalg.norm(k), 0.0):
        return np.eye(3)
    else:
        k = k / np.linalg.norm(k)
    
    K = _skew_symmetric(k)
    
    R = (
        np.eye(3) * np.cos(angle) 
        + K * np.sin(angle) 
        + np.outer(k, k) * (1 - np.cos(angle))
    )
    
    return R

In [ ]:
class Vector(np.ndarray):
    """
    A subclass of numpy.ndarray to represent 3D vector(s).
    
    This class allows for easy rotation of the vector(s).
    """
    
    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        if obj.shape[1] != 3:
            raise ValueError("Vector must be a 3D vector.")
        return obj
    
    def rotate(self, k, angle):
        """
        Rotate the vector around an axis k by a given angle.
        
        Parameters
        ----------
        k : array_like
            A 3D vector representing the axis of rotation.
        angle : float
            The angle in radians by which to rotate the vector.
        
        Returns
        -------
        Vector
            The rotated vector.

        Example
        -------
        >>> points = np.random.rand(10, 3)
        >>> k = np.array([0, 0, 1])
        >>> angle = np.pi / 4  # 45 degrees
        >>> rotated_points = Vector(points).rotate(k, angle)
        """
        R = rotation_matrix_axis(k, angle)
        return Vector((R @ self.T).T)

## 2 Tests

### 2.1 Rotate a point cloud

In [100]:
# test
x = np.linspace(-1, 1, 10)
y = np.linspace(-1, 1, 10)
z = np.linspace(-1, 1, 10)
x, y, z = np.meshgrid(x, y, z)
x = x.flatten()
y = y.flatten()
z = z.flatten()
points = Vector(np.stack([x, y, z], axis=-1))

In [101]:
k = np.array([0, 1, 0])
points_rotated = points.rotate(k, np.pi/4)

In [102]:
points_rotated

Vector([[-1.41421356e+00, -1.00000000e+00, -1.11022302e-16],
        [-1.25707872e+00, -1.00000000e+00,  1.57134840e-01],
        [-1.09994388e+00, -1.00000000e+00,  3.14269681e-01],
        ...,
        [ 1.09994388e+00,  1.00000000e+00, -3.14269681e-01],
        [ 1.25707872e+00,  1.00000000e+00, -1.57134840e-01],
        [ 1.41421356e+00,  1.00000000e+00,  1.11022302e-16]])

In [103]:
grid = pv.PolyData(points)
grid_rotated = pv.PolyData(points_rotated)
pl = pv.Plotter()
pl.add_mesh(grid, color="red")
pl.add_mesh(grid_rotated, color="green")
pl.add_lines(np.stack([-k, k]), color="blue")
pl.camera_position = "xy"
pl.show()

Widget(value='<iframe src="http://localhost:59501/index.html?ui=P_0x329eb8ac0_33&reconnect=auto" class="pyvist…

### 2.2 Rotate a vector

In this test, we rotate a vector by angles ranging from 0 to $2\pi$ to test the behavior of the rotation method.

In [104]:
v = np.array([[1, 1, 0]])
k = np.array([0, 10, 1])

In [105]:
pl = pv.Plotter()
pl.add_lines(np.stack([-k, k]), color="blue")
n_arrows = 10
cmap = plt.get_cmap("viridis", n_arrows)
for num, theta in enumerate(np.linspace(0, 2 * np.pi, 10, endpoint=False)):
    v = Vector(v)
    v_rotated = v.rotate(k, theta,)
    arrow = pv.Arrow(start=np.array([0,0,0]), direction=v_rotated, scale=0.5)
    pl.add_mesh(arrow, color=cmap(num), point_size=5, render_points_as_spheres=True)
pl.show()

Widget(value='<iframe src="http://localhost:59501/index.html?ui=P_0x30043bbe0_34&reconnect=auto" class="pyvist…

### 2.3 Rotate a flow field

The ultimate goal of the Vector class is to rotate the Oseen wake flow field. This involves rotating both the grid points and the associated velocity field. We test the rotation behavior in this section.

#### 2.3.1 The original flow 

In [106]:
from bounce import Bubble

In [107]:
a = 6e-4
U = 0.1

lim = 3*a
N = 10
bubble = Bubble(a, U)
x = np.linspace(-lim, lim, N)
y = np.linspace(-lim, lim, N)
z = np.linspace(-lim, lim, N)
x, y, z = np.meshgrid(x, y, z)
x = x.flatten()
y = y.flatten()
z = z.flatten()
points = Vector(np.stack([x, y, z], axis=-1))

In [108]:
flow = bubble.Oseen_wake(points)

In [116]:
grid = pv.PolyData(points)
grid["v"] = flow
glyph = grid.glyph(orient="v", scale="v", factor=0.02)
pl = pv.Plotter()
pl.show_axes()
pl.add_mesh(glyph)
pl.show()

Widget(value='<iframe src="http://localhost:59501/index.html?ui=P_0x3560a1700_41&reconnect=auto" class="pyvist…

#### 2.3.2 Rotated flow

In [117]:
k = np.array([0, 1, 0])
angle = np.pi / 4

points = Vector(points)
flow = Vector(flow)

points_rotated = points.rotate(k, angle)
flow_rotated = flow.rotate(k, angle)

In [118]:
grid = pv.PolyData(points_rotated)
grid["v"] = flow_rotated
glyph = grid.glyph(orient="v", scale="v", factor=0.02)
pl = pv.Plotter()
pl.show_axes()
pl.add_mesh(glyph)
pl.show()

Widget(value='<iframe src="http://localhost:59501/index.html?ui=P_0x302113f70_42&reconnect=auto" class="pyvist…